In [1]:
import math
import os
import ast

import sqlparse
import numpy as np
import pandas as pd
# from dotenv import load_dotenv
from sqlalchemy import create_engine, MetaData, Table, literal_column, Column
from sqlalchemy.sql import select, func, or_
from sqlalchemy.dialects.mysql import insert
from sqlalchemy.dialects.mysql.types import VARCHAR, TINYINT, TEXT

In [2]:
engine = create_engine("")

In [3]:
metadata = MetaData()
metadata.reflect(engine, schema="weaselbot")
# Table("regions", metadata, autoload_with=engine, schema="paxminer")

Table("regions", 
      metadata, 
      Column("region", VARCHAR(length=45), nullable=False, primary_key=True), 
      Column("slack_token", VARCHAR(length=90), nullable=False),
      Column("schema_name", VARCHAR(length=45), nullable=True, default=None),
      Column("active", TINYINT(), default=1),
      Column("firstf_channel", VARCHAR(length=45), nullable=True, default=None),
      Column("contact", VARCHAR(length=45), nullable=True, default=None),
      Column("send_pax_charts", TINYINT(), default=0),
      Column("send_ao_leaderboard", TINYINT(), default=0),
      Column("send_q_charts", TINYINT(), default=0),
      Column("send_region_leaderboard", TINYINT(), default=0),
      Column("send_region_uniquepax_chart", TINYINT(), default=0),
      Column("send_region_stats", VARCHAR(length=45), default=0),
      Column("send_mid_month_charts", VARCHAR(length=45), default=0),
      Column("comments", TEXT()),
      schema="paxminer");

In [4]:
cr = metadata.tables["weaselbot.combined_regions"]
cu = metadata.tables["weaselbot.combined_users"]
cud = metadata.tables["weaselbot.combined_users_dup"]
ca = metadata.tables["weaselbot.combined_aos"]
cb = metadata.tables["weaselbot.combined_beatdowns"]
catt = metadata.tables["weaselbot.combined_attendance"]

In [5]:
def insert_statement(table, insert_values, update_cols):
    sql = insert(table).values(insert_values)
    on_dup = sql.on_duplicate_key_update(
        {v.name: v for v in sql.inserted if v.name in update_cols}
    )
    return on_dup

In [6]:
def region_subquery(metadata):
    cb = metadata.tables["weaselbot.combined_beatdowns"]
    a = metadata.tables["weaselbot.combined_aos"]

    sql = select(
        a.c.region_id,
        func.max(cb.c.timestamp).label("max_timestamp"),
        func.max(cb.c.ts_edited).label("max_ts_edited"),
        func.count().label("beatdown_count"),
    )
    sql = sql.select_from(cb.join(a, cb.c.ao_id == a.c.ao_id))
    sql = sql.group_by(a.c.region_id).subquery("b")
    return sql

In [7]:
def paxminer_region_query(metadata):
    r = metadata.tables["paxminer.regions"]
    cr = metadata.tables["weaselbot.combined_regions"]
    sub = region_subquery(metadata)

    sql = select(
        r.c.schema_name,
        r.c.region.label("region_name"),
        sub.c.max_timestamp,
        sub.c.max_ts_edited,
        sub.c.beatdown_count,
        cr.c.region_id,
    )
    sql = sql.select_from(
        r.outerjoin(cr, r.c.schema_name == cr.c.schema_name).outerjoin(sub, cr.c.region_id == sub.c.region_id)
    )

    return sql

In [8]:
def weaselbot_region_query(metadata):
    cr = metadata.tables["weaselbot.combined_regions"]
    sub = region_subquery(metadata)

    sql = select(cr, sub.c.beatdown_count)
    sql = sql.select_from(cr.outerjoin(sub, cr.c.region_id == sub.c.region_id))

    return sql

In [9]:
paxminer_region_sql = paxminer_region_query(metadata) # verified
weaselbot_region_sql = weaselbot_region_query(metadata) # verified

In [10]:
### Sample data to move forward with ###

region_data = dict(schema_name="f3alamo f3chicago f3naperville f3omaha f3stcharles".split(),
                   region_name="Alamo Chicago Naperville Omaha St_Charles".split(),
                   max_timestamp=[1704824582.100129, 1704808955.413849, None, 1697642801.087519, 1704825215.550419],
                   max_ts_edited=[1704824731, 1704724809, None, None, 1704825356]
                  )
df_regions = pd.DataFrame(region_data)

In [11]:
insert_values = df_regions.to_dict("records")
update_cols = ("region_name", "max_timestamp", "max_ts_edited")
region_insert_sql = insert_statement(cr, insert_values, update_cols)

In [ ]:
# print(sqlparse.format(region_insert_sql.compile(engine, compile_kwargs={"literal_binds": True}).__str__(), keyword_case="upper", reindent=True))

In [12]:
dtypes = dict(
    region_id=pd.StringDtype(),  # this is a string everywhere else
    region_name=pd.StringDtype(),
    schema_name=pd.StringDtype(),
    slack_team_id=pd.StringDtype(),
    max_timestamp=pd.Float64Dtype(),
    max_ts_edited=pd.Float64Dtype(),
    beatdown_count=pd.Int16Dtype()
)

with engine.begin() as cnxn:
    df_regions = pd.read_sql(weaselbot_region_sql, cnxn, dtype=dtypes)

In [13]:
from f3_data_builder import pull_main_data, build_users, build_aos, build_beatdowns, build_attendance

In [14]:
df_users_dup, df_aos, df_beatdowns, df_attendance = pull_main_data(df_regions, engine, metadata)

starting f3alamo... Done
starting f3albany... Done
starting f3albuquerque... Done
starting f3alliance... Done
starting f3anderson... Done
starting f3annapolis... Done
starting f3atlanta... Done
starting f3austin... Done
starting f3badlands... Done
starting f3beast... Done
starting f3bellingham... Done
starting f3blueridge... Done
starting f3borderlands... Done
starting f3brentwood... Done
starting f3bruco... Done
starting f3youngsville... Done
starting f3carpex... Done
starting f3centralarkansas... Done
starting f3central_il... Done
starting f3charleston... Done
starting f3charlottesville... Done
starting f3cherokee... Done
starting f3cheyenne... Done
starting f3chicago... Done
starting f3churham... Done
starting f3clearwater... Done
starting f3cleburne... Done
starting f3cleveland... Done
starting f3coloradosprings... Done
starting f3columbia... Done
starting f3columbus... Done
starting f3dallas... Done
starting f3davidson... Done
starting f3dayton... Done
starting f3delhi_ncr... Done

In [15]:
print(f"beatdowns to process: {len(df_beatdowns)}")

beatdowns to process: 75937


In [16]:
df_users_dup = build_users(df_users_dup, df_attendance, engine, metadata)

building users...


In [17]:
df_aos = build_aos(df_aos, engine, metadata)

building aos...


In [18]:
df_beatdowns = build_beatdowns(df_beatdowns, df_users_dup, df_aos, engine, metadata)

building beatdowns...


In [19]:
build_attendance(df_attendance, df_users_dup, df_aos, df_beatdowns, engine, metadata)

building attendance...


In [ ]:
#### PULLING USER TABLE DATA. TEST LOGIC ON JUST ONE

In [ ]:
df_users_dup_list, df_aos_list, df_beatdowns_list, df_attendance_list = [], [], [], []
users_dtypes = dict(
    slack_user_id=pd.StringDtype(),
    user_name=pd.StringDtype(),
    email=pd.StringDtype(),
    region_id=pd.StringDtype()
)
ao_dtypes = dict(
    slack_channel_id=pd.StringDtype(),
    ao_name=pd.StringDtype(),
    region_id=pd.StringDtype()
)
beatdown_dtypes = dict(
    slack_channel_id=pd.StringDtype(),
    slack_q_user_id=pd.StringDtype(),
    slack_coq_user_id=pd.StringDtype(),
    pax_count=pd.Int16Dtype(),
    fng_count=pd.Int16Dtype(),
    region_id=pd.StringDtype(),
    timestamp=pd.Float64Dtype(),
    ts_edited=pd.StringDtype(), # string for now. Then replace "NA" string and cast as float
    backblast=pd.StringDtype(),
    json=pd.StringDtype()
)
attendance_dtypes = dict(
    slack_channel_id=pd.StringDtype(),
    slack_q_user_id=pd.StringDtype(),
    slack_user_id=pd.StringDtype(),
    region_id=pd.StringDtype(),
    json=pd.StringDtype()
)

In [ ]:
for row in df_regions.itertuples(index=False):
    if row.schema_name == "f3chicago":
        db = row.schema_name
        usr = Table("users", metadata, autoload_with=engine, schema=db)
        beatdowns = Table("beatdowns", metadata, autoload_with=engine, schema=db)
        attendance = Table("bd_attendance", metadata, autoload_with=engine, schema=db)
        ao = Table("aos", metadata, autoload_with=engine, schema=db)
        
        user_sql = select(
            usr.c.user_id.label("slack_user_id"),
            usr.c.user_name,
            usr.c.email,
            literal_column(f"'{row.region_id}'").label("region_id"),
        )
        
        aos_sql = select(
            ao.c.channel_id.label("slack_channel_id"),
            ao.c.ao.label("ao_name"),
            literal_column(f"'{row.region_id}'").label("region_id"),
        )
        
        beatdowns_base_sql = select(beatdowns.c.ao_id.label("slack_channel_id"),
                               beatdowns.c.bd_date,
                               beatdowns.c.q_user_id.label("slack_q_user_id"),
                               beatdowns.c.coq_user_id.label("slack_coq_user_id"),
                               beatdowns.c.pax_count,
                               beatdowns.c.fng_count,
                               literal_column(f"'{row.region_id}'").label("region_id"),
                               beatdowns.c.timestamp,
                               beatdowns.c.ts_edited,
                               beatdowns.c.backblast,
                               beatdowns.c.json)
        beatdowns_sql = beatdowns_base_sql.where(or_(beatdowns.c.timestamp > str(row.max_timestamp),
                                                beatdowns.c.ts_edited > str(row.max_ts_edited)))
        
        attendance_base_sql = select(attendance.c.ao_id.label("slack_channel_id"),
                                 attendance.c.date.label("bd_date"),
                                 attendance.c.q_user_id.label("slack_q_user_id"),
                                 attendance.c.user_id.label("slack_user_id"),
                                 literal_column(f"'{row.region_id}'").label("region_id"),
                                 attendance.c.json)
        attendance_sql = attendance_base_sql.where(or_(attendance.c.timestamp > str(row.max_timestamp),
                                                attendance.c.ts_edited > str(row.max_ts_edited)))
        
        beatdowns_no_ts_sql = beatdowns_base_sql
        attendance_no_ts_sql = attendance_base_sql
        beatdowns_no_ed_sql = beatdowns_base_sql.where(beatdowns.c.timestamp > str(row.max_timestamp))
        attendance_no_ed_sql = attendance_base_sql.where(attendance.c.timestamp > str(row.max_timestamp))

        with engine.begin() as cnxn:
            df_users_dup_list.append(pd.read_sql(user_sql, cnxn, dtype=users_dtypes))
            df_aos_list.append(pd.read_sql(aos_sql, cnxn, dtype=ao_dtypes))
            if (not math.isnan(row.max_timestamp)) and (not math.isnan(row.max_ts_edited)):
                df_beatdowns_list.append(pd.read_sql(beatdowns_sql, cnxn, parse_dates="bd_date", dtype=beatdown_dtypes))
                df_attendance_list.append(pd.read_sql(attendance_sql, cnxn, parse_dates="bd_date", dtype=attendance_dtypes))
            elif not math.isnan(row.max_timestamp):
                df_beatdowns_list.append(pd.read_sql(beatdowns_no_ed_sql, cnxn, parse_dates="bd_date", dtype=beatdown_dtypes))
                df_attendance_list.append(pd.read_sql(attendance_no_ed_sql, cnxn, parse_dates="bd_date", dtype=attendance_dtypes))
            elif row.beatdown_count == 0:
                df_beatdowns_list.append(pd.read_sql(beatdowns_no_ts_sql, cnxn, parse_dates="bd_date", dtype=beatdown_dtypes))
                df_attendance_list.append(pd.read_sql(attendance_no_ts_sql, cnxn, parse_dates="bd_date", dtype=attendance_dtypes))

In [ ]:
# print(sqlparse.format(beatdowns_no_ed_sql.compile(engine, compile_kwargs={'literal_binds': True}).__str__(), keyword_case='upper', reindent=True))

In [ ]:
df_users_dup = pd.concat(df_users_dup_list)
df_aos = pd.concat(df_aos_list)
df_beatdowns = pd.concat(df_beatdowns_list)
df_attendance = pd.concat(df_attendance_list)

df_beatdowns.ts_edited = df_beatdowns.ts_edited.replace("NA", pd.NA).astype(pd.Float64Dtype())

In [ ]:
# df_beatdowns # ts_edited has None and "NA" values. json has None and {}. Not good
# df_attendance_list[0]

In [ ]:
df_users_dup["email"] = df_users_dup["email"].str.lower()
df_users_dup = df_users_dup[df_users_dup["email"].notna()]

df_user_agg = (
    df_attendance.groupby(["slack_user_id"], as_index=False)["bd_date"].count().rename({"bd_date": "count"}, axis=1)
)
df_users = (
    df_users_dup.merge(df_user_agg[["slack_user_id", "count"]], on="slack_user_id", how="left")
    .fillna(0)
    .sort_values(by="count", ascending=False)
)


In [ ]:
# I don't like this. I think we should clean up our info.
# We have two unique PAX named Gump
# Amtrak has one account disabled
# Delete all pending Slack invitations 2024-01-13

df_users.drop_duplicates(subset=["email"], keep="first", inplace=True)

In [ ]:
# x = df_users.groupby("user_name")["email"].agg(len)

# df_users.loc[df_users.user_name.isin(x[x>1].index.values)].sort_values("user_name")

In [ ]:
insert_values = df_users[["user_name", "email", "region_id"]].rename({"region_id": "home_region_id"}, axis=1).to_dict("records")
update_cols = ("user_name", "email", "home_region_id")
user_insert_sql = insert_statement(cu, insert_values, update_cols)

# print(sqlparse.format(user_insert_sql.compile(engine, compile_kwargs={'literal_binds': True}).__str__(), keyword_case='upper', reindent=True))

In [ ]:
df_users_dup.loc[df_users_dup.user_id.isna()]

In [ ]:
dtypes = dict(
    user_id=pd.StringDtype(), # executive decision to use String instead of Int
    user_name=pd.StringDtype(),
    email=pd.StringDtype(),
    home_region_id=pd.StringDtype() # executive decision to use String instead of Int
)
df_users = pd.read_sql(select(cu), engine, dtype=dtypes)
df_users_dup = df_users_dup.merge(df_users[["email", "user_id"]], on="email", how="left")

insert_values = df_users_dup[["slack_user_id", "user_name", "email", "region_id", "user_id"]].to_dict("records")

for d in insert_values:
    try:
        d["user_id"] = int(d["user_id"])
    except TypeError:
        pass # These are NA values being allowed to pass through

update_cols = ("user_name", "email", "region_id", "user_id")
user_dup_insert_sql = insert_statement(cud, insert_values, update_cols)

# print(sqlparse.format(user_dup_insert_sql.compile(engine, compile_kwargs={'literal_binds': True}).__str__(), keyword_case='upper', reindent=True))

In [ ]:
insert_values = df_aos[["slack_channel_id", "ao_name", "region_id"]].to_dict("records")
update_cols = ("ao_name",)
aos_insert_sql = insert_statement(ca, insert_values, update_cols)

# print(sqlparse.format(aos_insert_sql.compile(engine, compile_kwargs={'literal_binds': True}).__str__(), keyword_case='upper', reindent=True))

In [ ]:
dtypes = {"ao_id": pd.StringDtype(), # executive decision to use String instead of Int
          "slack_channel_id": pd.StringDtype(),
          "ao_name": pd.StringDtype(),
          "region_id": pd.StringDtype(), # int in the SQL table, but joins with others are strings.
         }
df_aos = pd.read_sql(select(ca), engine, dtype=dtypes)

In [ ]:
### beatdowns ###

In [ ]:
def extract_user_id(slack_user_id):
    match isinstance(slack_user_id, type(pd.NA)):
        case True:
            return pd.NA
        case _:    
            if slack_user_id.startswith("U"):
                return slack_user_id
            elif "team" in slack_user_id:
                return slack_user_id.split("/team/")[1].split("|")[0]
            else:
                pass

In [ ]:
df_beatdowns["slack_q_user_id"] = df_beatdowns["slack_q_user_id"].apply(extract_user_id).astype(pd.StringDtype())
df_beatdowns["slack_coq_user_id"] = df_beatdowns["slack_coq_user_id"].apply(extract_user_id).astype(pd.StringDtype())

In [ ]:
df_beatdowns = (df_beatdowns
                .merge(df_users_dup[["slack_user_id", "user_id", "region_id"]], 
                       left_on=["slack_q_user_id", "region_id"],
                       right_on=["slack_user_id", "region_id"], 
                       how="left")
                .rename({"user_id": "q_user_id"}, axis=1)
               .merge(df_users_dup[["slack_user_id", "user_id", "region_id"]],
                       left_on=["slack_coq_user_id", "region_id"],
                       right_on=["slack_user_id", "region_id"],
                       how="left",)
                .rename({"user_id": "coq_user_id"}, axis=1)
 .merge(
    df_aos[["slack_channel_id", "ao_id", "region_id"]],
    on=["slack_channel_id", "region_id"],
    how="left",
)
)
df_beatdowns["fng_count"] = df_beatdowns["fng_count"].fillna(0) # no need to cast as int. Maybe still need fillna. unsure

In [ ]:
# ast.literal_eval(
# for _, d in enumerate(insert_values):
#     if d["json"] is not None:
#         print(_)

In [ ]:
insert_values = df_beatdowns[df_beatdowns["ao_id"].notna()][
    [
        "ao_id",
        "bd_date",
        "q_user_id",
        "coq_user_id",
        "pax_count",
        "fng_count",
        "timestamp",
        "ts_edited",
        "backblast",
        "json",
    ]
].to_dict("records")
#.replace(np.nan, None).to_dict("records")
# insert_values

# below columns are INT in their target table. coerce them so they'll load properly
for d in insert_values:
    for col in ("ao_id", "q_user_id", "coq_user_id"):
        try:
            d[col] = int(d[col])
        except TypeError:
            pass
    if d["json"] is not None:
        d["json"] = ast.literal_eval(d["json"]) # on the fence. This may not work

In [ ]:
update_cols = ("coq_user_id", "pax_count", "fng_count", "timestamp", "ts_edited", "backblast", "json")

beatdowns_insert_sql = insert_statement(cb, insert_values, update_cols)
# print(
#     sqlparse.format(
#         beatdowns_insert_sql.compile(engine, compile_kwargs={'literal_binds': True})
#         .__str__(), keyword_case='upper', reindent=True)
# )

In [ ]:
dtypes = dict(beatdown_id=pd.StringDtype(),
              ao_id=pd.StringDtype(),
              q_user_id=pd.StringDtype(),
              coq_user_id=pd.StringDtype(),
              pax_count=pd.Int16Dtype(),
              fng_count=pd.Int16Dtype(),
              timestamp=pd.Float64Dtype(),
              ts_edited=pd.Float64Dtype(),
              backblast=pd.StringDtype(),
              json=pd.StringDtype(),
             )
df_beatdowns = pd.read_sql(select(cb), engine, parse_dates="bd_date", dtype=dtypes)
df_beatdowns.q_user_id = df_beatdowns.q_user_id.astype(pd.Float64Dtype()).astype(pd.Int64Dtype()).astype(pd.StringDtype())
# need to cast q_user_id as above. In the data it's integer.
# pandas reads it as a float with ".0" at the end. This trims that so the subsequent pandas joins work

In [ ]:
### Attendance ###

In [ ]:
df_attendance["slack_user_id"] = df_attendance["slack_user_id"].apply(extract_user_id).astype(pd.StringDtype())
df_attendance["slack_q_user_id"] = df_attendance["slack_q_user_id"].apply(extract_user_id).astype(pd.StringDtype())
df_attendance = (df_attendance.merge(
    df_users_dup[["slack_user_id", "user_id", "region_id"]],
    left_on=["slack_q_user_id", "region_id"],
    right_on=["slack_user_id", "region_id"],
    how="left",
).rename({"user_id": "q_user_id", "slack_user_id_x": "slack_user_id"}, axis=1)
 .drop("slack_user_id_y", axis=1)
).merge(
    df_users_dup[["slack_user_id", "user_id", "region_id"]],
    on=["slack_user_id", "region_id"],
    how="left",
).merge(
    df_aos[["slack_channel_id", "ao_id", "region_id"]],
    on=["slack_channel_id", "region_id"],
    how="left",
).merge(
    df_beatdowns[["beatdown_id", "bd_date", "q_user_id", "ao_id"]],
    on=["bd_date", "q_user_id", "ao_id"],
    how="left",
)

df_attendance.drop_duplicates(subset=["beatdown_id", "user_id"], inplace=True)
df_attendance = df_attendance[df_attendance["beatdown_id"].notnull()]
df_attendance = df_attendance[df_attendance["user_id"].notnull()]

In [ ]:
insert_values = df_attendance[["beatdown_id", "user_id", "json"]].to_dict("records")
update_cols = ("beatdown_id", "json")
attendance_insert_sql = insert_statement(catt, insert_values, update_cols)

In [ ]:
### region ###

In [ ]:
with engine.connect() as cnxn:
    df_regions = pd.read_sql(paxminer_region_sql, cnxn)
    insert_values = df_regions[["schema_name", "region_name", "max_timestamp", "max_ts_edited"]].to_dict("records")
    update_cols = ("region_name", "max_timestamp", "max_ts_edited")
    region_insert_sql = insert_statement(cr, insert_values, update_cols)